In [2]:
import nltk
from nltk.corpus import stopwords
import random
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.utils import shuffle
import string
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
def find_features(top_1000_words, text):
    feature = {}
    for word in top_1000_words:
        feature[word] = word in text.lower()
    return feature

In [4]:
df = pd.read_csv('gender-classifier-DFE-791531.csv')
df = shuffle(shuffle(shuffle(df)))
df

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
15932,815735433,False,finalized,3,10/26/15 21:58,female,1.0000,yes,1.0000,1/1/13 17:46,...,https://pbs.twimg.com/profile_images/657671059...,0,C0DEED,When you stayed up until 2:00 am finishing hom...,NaN,8216,10/26/15 13:19,6.587400e+17,NaN,Atlantic Time (Canada)
8056,815727378,False,finalized,3,10/27/15 0:11,female,1.0000,yes,1.0000,3/8/14 15:54,...,https://pbs.twimg.com/profile_images/654928516...,0,C0DEED,The Impudence of Rabiu Kwakwanso and his Fulan...,NaN,19856,10/26/15 12:39,6.587300e+17,NaN,Pacific Time (US & Canada)
2829,815722123,False,finalized,3,10/27/15 0:21,male,1.0000,yes,1.0000,12/8/13 13:49,...,https://pbs.twimg.com/profile_images/618791430...,0,C0DEED,@BemaniStar they aren't even putting the machi...,NaN,923,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
18361,815737917,False,finalized,3,10/26/15 23:05,female,1.0000,yes,1.0000,6/21/11 10:48,...,https://pbs.twimg.com/profile_images/568867831...,0,0,and of course coping with the fucking stress a...,NaN,287,10/26/15 13:19,6.587400e+17,NaN,NaN
11817,815731214,False,finalized,3,10/27/15 1:51,brand,1.0000,yes,1.0000,9/15/15 17:59,...,https://pbs.twimg.com/profile_images/643952775...,0,C0DEED,5 Fruits and Vegetables That Look Suspiciously...,NaN,12181,10/26/15 13:20,6.587400e+17,"Follow her for inspiration,",NaN
1410,815720647,False,finalized,3,10/27/15 1:25,male,0.6759,yes,1.0000,9/23/09 21:32,...,https://pbs.twimg.com/profile_images/592388799...,0,C0DEED,@vfinch @sternshow the show be called Howard a...,NaN,15945,10/26/15 12:40,6.587300e+17,buffalo new york,NaN
2379,815721671,False,finalized,3,10/26/15 23:33,female,1.0000,yes,1.0000,7/19/11 13:47,...,https://pbs.twimg.com/profile_images/648353867...,0,FFFFFF,The elevator came and this girl got off on acc...,NaN,16388,10/26/15 12:40,6.587300e+17,atlantis,Eastern Time (US & Canada)
10820,815730192,False,finalized,3,10/26/15 23:05,male,0.6635,yes,1.0000,7/30/11 8:46,...,https://pbs.twimg.com/profile_images/636010048...,0,C0DEED,I went on a trip and came home to discover it ...,NaN,15266,10/26/15 13:20,6.587400e+17,NYC / Lima,Pacific Time (US & Canada)
16080,815735583,False,finalized,3,10/26/15 23:59,female,1.0000,yes,1.0000,11/4/10 2:12,...,https://pbs.twimg.com/profile_images/496093986...,0,181A1E,"@neyonx02 I've had all of them, I just wanted ...",NaN,1571,10/26/15 13:19,6.587400e+17,NaN,Edinburgh
6301,815725612,False,finalized,3,10/27/15 0:02,brand,1.0000,yes,1.0000,8/25/14 12:23,...,https://abs.twimg.com/sticky/default_profile_i...,0,C0DEED,Get Weather Updates from The Weather Channel. ...,NaN,36527,10/26/15 12:40,6.587300e+17,NaN,NaN


In [5]:
all_descriptions = df['description']
all_tweets = df['text']
all_genders = df['gender']
all_gender_confidence = df['gender:confidence']
description_tweet_gender = []

In [6]:
# Creation of bag of words for the description
bag_of_words = []
c = 0
stop = stopwords.words('english')
for tweet in all_tweets:
    description = all_descriptions[c]
    gender = all_genders[c]
    gender_confidence = all_gender_confidence[c]
    if (str(tweet) == 'nan' and str(description) == 'nan') or str(gender) == 'nan' or str(gender) == 'unknown' or float(gender_confidence) < 0.8:
        c+=1
        continue
    
    if str(tweet) == 'nan':
        tweet = ''
    if str(description) == 'nan':
        description = ''
    
    # removal of punctuations
    for punct in string.punctuation:
        if punct in tweet:
            tweet = tweet.replace(punct, " ")
        if punct in description:
            description = description.replace(punct, " ")
            
    # adding the word to the bag
    for word in tweet.split():
        if word.isalpha() and word.lower() not in stop:
            bag_of_words.append(word.lower())
    for word in description.split():
        if word.isalpha() and word.lower() not in stop:
            bag_of_words.append(word.lower())
    
    description_tweet_gender.append((tweet+" "+description , gender))
    c += 1

print(len(bag_of_words))
print(len(description_tweet_gender))

232247
13817


In [25]:
# get top 1000 words which will act as our features of each sentence
bag_of_words = nltk.FreqDist(bag_of_words)
top_1000_words = []
for word in bag_of_words.most_common(2000):
    top_1000_words.append(word[0])

top_1000_words

['co',
 'https',
 'weather',
 'get',
 'love',
 'updates',
 'like',
 'channel',
 'http',
 'one',
 'life',
 'new',
 'news',
 'time',
 'best',
 'people',
 'day',
 'world',
 'follow',
 'music',
 'go',
 'know',
 'good',
 'us',
 'make',
 'see',
 'want',
 'need',
 'back',
 'amp',
 'live',
 'got',
 'business',
 'last',
 'fan',
 'free',
 'twitter',
 'year',
 'great',
 'work',
 'still',
 'think',
 'way',
 'always',
 'u',
 'would',
 'sports',
 'things',
 'god',
 'first',
 'today',
 'girl',
 'com',
 'man',
 'writer',
 'home',
 'never',
 'come',
 'right',
 'also',
 'im',
 'going',
 'makes',
 'really',
 'real',
 'much',
 'game',
 'via',
 'even',
 'media',
 'lover',
 'help',
 'tweets',
 'family',
 'find',
 'happy',
 'say',
 'video',
 'artist',
 'check',
 'please',
 'take',
 'let',
 'old',
 'instagram',
 'look',
 'every',
 'ig',
 'social',
 'food',
 'account',
 'health',
 'marketing',
 'everything',
 'made',
 'well',
 'big',
 'ever',
 'could',
 'show',
 'better',
 'latest',
 'around',
 'week',
 'tweet

In [32]:
# creating the feature set, training set and the testing set
feature_set = [(find_features(top_1000_words, text), gender) for (text, gender) in description_tweet_gender]
training_set = feature_set[:int(len(feature_set)*3/4)]
testing_set = feature_set[int(len(feature_set)*3/4):]

print("Length of feature set", len(feature_set))
print("Length of training set", len(training_set))
print("Length of testing set", len(testing_set))

Length of feature set 13817
Length of training set 10362
Length of testing set 3455


In [33]:
# creating a naive bayes classifier
NB_classifier = nltk.NaiveBayesClassifier.train(training_set)
accuracy = nltk.classify.accuracy(NB_classifier, testing_set)*100
print("Naive Bayes Classifier accuracy = " + str(accuracy))
NB_classifier.show_most_informative_features(20)

Naive Bayes Classifier accuracy = 56.35311143270623
Most Informative Features
              continuous = True            brand : female =     34.1 : 1.0
               promoting = True            brand : female =     28.6 : 1.0
               empowered = True            brand : female =     25.3 : 1.0
               providing = True            brand : male   =     24.1 : 1.0
                  proves = True            brand : male   =     23.3 : 1.0
                 outlook = True            brand : female =     22.0 : 1.0
                feminist = True           female : brand  =     21.7 : 1.0
                 inquiry = True            brand : female =     20.3 : 1.0
                    quiz = True            brand : male   =     19.5 : 1.0
               geography = True            brand : female =     19.5 : 1.0
               dedicated = True            brand : male   =     18.0 : 1.0
           professionals = True            brand : female =     17.0 : 1.0
                  secu

In [34]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
accuracy = nltk.classify.accuracy(MNB_classifier, testing_set)*100
print("Multinomial Naive Bayes Classifier accuracy =", (accuracy))

MemoryError: 

In [35]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
accuracy = nltk.classify.accuracy(LogisticRegression_classifier, testing_set)*100
print("Logistic Regression classifier accuracy =", accuracy)

MemoryError: 